In [2]:
import os
import pandas as pd

reviews_file = 'src/reviews.csv'
if os.path.exists(reviews_file):
    reviews_df = pd.read_csv(reviews_file)
else:
    reviews_df = pd.DataFrame(columns=["user_id", "restaurant_name", "review_text", "rating"])
    

In [3]:
reviews_df

,user_id,restaurant_name,review_text,rating


In [6]:
review_data = {
            "user_id": 'user_id',
            "restaurant_name": 'selected_restaurant',
            "review_text": 'review_text',
            "rating": 'rating',
    }

In [7]:
review_data

{'user_id': 'user_id',
 'restaurant_name': 'selected_restaurant',
 'review_text': 'review_text',
 'rating': 'rating'}

In [4]:
        # Append the new review to the dataframe
reviews_df = reviews_df.append(review_data, ignore_index=True)

AttributeError: 'DataFrame' object has no attribute 'append'

In [4]:
from typing import List
import pickle
from collections import defaultdict
import random
import os

import requests
import joblib
import pandas as pd
import numpy as np
from PIL import Image

In [16]:
folder_path = "../images"
restaurants = pickle.load(open('../artifacts/restaurants_list.pkl','rb'))
similarity = pickle.load(open('../artifacts/restaurants_similarity.pkl','rb'))
cf_model = joblib.load('../models/cf_model.pkl')

In [17]:
restaurants

,restaurant_id,restaurant_name,cuisine_type,location,price_range,average_rating,num_of_reviews,popular_dishes,operating_hours,ambiance,special_features,tags
0,R000,Mija's Pasta,Asian,Osogbo,4000-12000,[3.7],929,[Pasta],"[10AM, 8PM]",[FamilyFriendly],[WheelchairAccessible],asian osogbo 4000 12000 3.7 pasta 10am 8pm fam...
1,R001,FishVillae,Indian,Ede,29000-47000,[2.4],335,"[Boiled, Yam, with, Egg, Sauce]","[10AM, 9PM]",[Casual],"[KidFriendly, WheelchairAccessible]",indian ede 29000 47000 2.4 boil yam with egg s...
2,R002,Jo's Indulge,Italian,Ife,20000-44000,[4.5],637,"[Chicken, and, Chips]","[10AM, 9PM]",[FamilyFriendly],"[WheelchairAccessible, LiveMusic, KidFriendly]",italian ife 20000 44000 4.5 chicken and chip 1...
3,R003,New Image african kitchen,Indian,Ife,3000-10000,[1.9],388,"[Amala, with, Gbegiri, and, Ewedu, Soup]","[10AM, 9PM]",[FamilyFriendly],"[PetFriendly, WheelchairAccessible, OutdoorSea...",indian ife 3000 10000 1.9 amala with gbegiri a...
4,R004,Yakoyo Afrikana,Italian,Ife,4000-12000,[3.5],405,"[Pounded, Yam, and, Egusii]","[9AM, 10PM]",[FamilyFriendly],"[PetFriendly, OutdoorSeating]",italian ife 4000 12000 3.5 pound yam and egusi...
...,...,...,...,...,...,...,...,...,...,...,...,...
78,R078,Abiola Beancake,American,Ife,27000-42000,[1.6],420,"[Akara, and, Pap]","[9AM, 9PM]",[FamilyFriendly],[WheelchairAccessible],american ife 27000 42000 1.6 akara and pap 9am...
79,R079,Casa De Fries,American,Ife,3000-15000,[3.5],835,"[Chicken, and, Chips]","[9AM, 8PM]",[Casual],"[OutdoorSeating, WheelchairAccessible, KidFrie...",american ife 3000 15000 3.5 chicken and chip 9...
80,R080,Indeego Restaurant &Lounge,Indian,Ife,27000-43000,[1.4],537,"[Fried, Rice, and, Chicken]","[9AM, 9PM]",[Romantic],"[OutdoorSeating, WheelchairAccessible, KidFrie...",indian ife 27000 43000 1.4 fri rice and chicke...
81,R081,Jopa Foodies,Asian,Ife,10000-30000,[4.8],307,"[Fried, Rice, and, Chicken]","[10AM, 10PM]",[FastFood],"[LiveMusic, OutdoorSeating]",asian ife 10000 30000 4.8 fri rice and chicken...


In [14]:
restaurants[['cuisine_type','location','price_range']]

,cuisine_type,location,price_range
0,Asian,Osogbo,4000-12000
1,Indian,Ede,29000-47000
2,Italian,Ife,20000-44000
3,Indian,Ife,3000-10000
4,Italian,Ife,4000-12000
...,...,...,...
78,American,Ife,27000-42000
79,American,Ife,3000-15000
80,Indian,Ife,27000-43000
81,Asian,Ife,10000-30000


In [11]:
# Extract the first item from each list in the columns
restaurants['cuisine_type'] = restaurants['cuisine_type'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)
restaurants['location'] = restaurants['location'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)
restaurants['price_range'] = restaurants['price_range'].apply(lambda x: x if not isinstance(x, list) else [x[0], x[1]] if len(x) == 2 else x)


In [13]:
restaurants['price_range'] = restaurants['price_range'].apply(
    lambda x: f"{x[0]}-{x[1]}" if isinstance(x, list) and len(x) == 2 else x
)


In [15]:
restaurants.to_pickle('../artifacts/restaurants_list.pkl')

In [18]:
USER_IDS = [f'U{str(i).zfill(3)}' for i in range(100)]
user_id = random.choice(USER_IDS)


folder_path = "../images"
restaurants = pickle.load(open('../artifacts/restaurants_list.pkl','rb'))
similarity = pickle.load(open('../artifacts/restaurants_similarity.pkl','rb'))
cf_model = joblib.load('../models/cf_model.pkl')


# Extract the first item from each list in the columns
restaurants['cuisine_type'] = restaurants['cuisine_type'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)
restaurants['location'] = restaurants['location'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)
restaurants['price_range'] = restaurants['price_range'].apply(lambda x: x if not isinstance(x, list) else [x[0], x[1]] if len(x) == 2 else x)

restaurants['price_range'] = restaurants['price_range'].apply(
    lambda x: f"{x[0]}-{x[1]}" if isinstance(x, list) and len(x) == 2 else x
)


# Function to load images from a folder
def load_images_from_folder(image_name: str, folder_path: str = folder_path):
    img_path = os.path.join(folder_path, image_name)
    image = Image.open(img_path)
    return image


def fetch_restaurant_poster(restaurant_id):
    restaurant_poster = load_images_from_folder(f"{restaurant_id}.png")
    return restaurant_poster


In [19]:
def get_top_n(predictions, n=6):
    top_n = defaultdict(list)
    for uid, iid, est in predictions:
        top_n[uid].append([iid, est])

    # Sort the predictions for each user and retrieve the highest `n` ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [31]:
filtered_recommendations


defaultdict(list, {'U035': []})

In [33]:
filtered_recommendations = run_collabotive_filtering(
    user_id=user_id, 
    restaurant_ids=['R001', 'R002', 'R003'], 
    cuisine_type="Italian", 
    location="Downtown", 
    price_range="28000-47000"
)

print(filtered_recommendations)


defaultdict(<class 'list'>, {'U035': []})


In [28]:
filtered_recommendations


defaultdict(list, {'U035': []})